In [32]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime
import env
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
def run_query(sql_name):
    # Connect to the MySQL database (replace the placeholders with your actual credentials)
    host = '10.48.103.23'
    user = 't_2023_user'        
    password = '99tpZBfpXn'
    database = 'master'
    port = 33144

    # Create a connection to the database
    connection = mysql.connector.connect(host=host, user=user, password=password, database=database, port=port)

    # Create a cursor object to execute the SQL query
    cursor = connection.cursor()

    # Define the SQL query
    current_dir = f"./"
    file_path = f'{current_dir}{sql_name}'

    # Read the content of the file into a variable
    with open(file_path, 'r') as file:
        query = file.read()

    # Execute the query
    cursor.execute(query)

    # Fetch all the results into a list of tuples
    results = cursor.fetchall()

    # Get the column names from the cursor description
    columns = [col[0] for col in cursor.description]

    # Close the cursor and the connection
    cursor.close()
    connection.close()

    # Create a DataFrame from the results and column names
    df = pd.DataFrame(results, columns=columns)
    return df

# mager maka auto hapus yg korelasi tinggi
def remove_highly_correlated_features(df, threshold=0.9):
    """
    Menghapus fitur yang memiliki korelasi lebih dari ambang batas tertentu.
    
    Parameters:
    - df: DataFrame yang berisi fitur-fitur yang akan diperiksa.
    - threshold: Ambang batas korelasi.
    
    Returns:
    - DataFrame yang telah dikurangi fiturnya.
    """
    # Hanya mengambil kolom numerik untuk korelasi
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr().abs()
    
    # Dapatkan matriks segitiga atas dari matriks korelasi
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    
    #print(f"kolom dengan korelasi di atas {threshold}: {upper_triangle.columns}")
    
    # Temukan indeks kolom fitur yang memiliki korelasi lebih dari ambang batas
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
    
    return df.drop(columns=to_drop)

def manual_oversample(X, y):
    # Dapatkan indeks dari kelas minoritas
    minority_class_indices = y[y == 1].index
    majority_class_indices = y[y == 0].index
    
    # Hitung jumlah kelas mayoritas
    num_majority = len(y[y == 0])
    
    # Dapatkan sampel acak dari kelas minoritas
    random_minority_indices = np.random.choice(minority_class_indices, num_majority, replace=True)
    
    # Gabungkan indeks kelas mayoritas dengan sampel acak dari kelas minoritas
    over_sample_indices = np.concatenate([majority_class_indices, random_minority_indices])
    
    # Dapatkan data yang oversampled
    X_oversampled = X.loc[over_sample_indices]
    y_oversampled = y.loc[over_sample_indices]
    
    return X_oversampled, y_oversampled

In [40]:
#download dataset
download_df = run_query("eda/DATASET_CASH_BALANCE.sql")
print("downloading...")
# download_df.set_index('reco_id_curr')
download_df.to_csv("eda/DATASET_CASH_BALANCE.csv")
print("csv generated")

downloading...
csv generated


In [43]:
download_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480323 entries, 0 to 480322
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   reco_id_last              480323 non-null  object 
 1   reco_id_curr              480323 non-null  object 
 2   months_balance            480323 non-null  int64  
 3   installment_count         480323 non-null  float64
 4   installment_future_count  480323 non-null  float64
 5   contract_status_name      480323 non-null  object 
 6   reco_dpd                  480323 non-null  int64  
 7   reco_dpd_def              480323 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 29.3+ MB


In [34]:
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [36]:
df = pd.read_csv('eda/DATASET_BKI.csv')
df.drop(df.filter(regex='Unnamed').columns, axis=1, inplace=True)
target_df = pd.read_csv('dataset/DATASET_TARGET.csv')
df = df.merge(target_df, on='reco_id_curr', how='right')
df.describe()

,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,credit_limit_max_overdue,credit_prolong_count,credit_sum,credit_sum_debt,credit_sum_limit,credit_sum_overdue,days_credit_update,annuity_payment,target
count,338866.000000,338866.00000,338866.000000,338866.000000,3.388660e+05,338866.000000,3.388660e+05,3.388660e+05,3.388660e+05,3.388660e+05,338866.000000,3.388660e+05,372960.000000
mean,-1140.075304,0.95034,437.351773,-643.608943,1.182476e+03,0.007144,3.140744e+05,1.027263e+05,4.060766e+03,3.070675e+01,-601.584809,3.477316e+03,0.078132
std,799.381575,40.17249,4843.039939,750.419832,3.712291e+04,0.101828,7.859205e+05,5.535188e+05,3.421420e+04,5.562517e+03,728.763391,1.204669e+05,0.268379
min,-2922.000000,0.00000,-42060.000000,-2919.000000,0.000000e+00,0.000000,0.000000e+00,-1.354880e+06,-3.163920e+05,0.000000e+00,-41946.000000,0.000000e+00,0.000000
25%,-1663.000000,0.00000,-1089.000000,-1084.000000,0.000000e+00,0.000000,4.912650e+04,0.000000e+00,0.000000e+00,0.000000e+00,-907.000000,0.000000e+00,0.000000
50%,-985.000000,0.00000,-268.000000,-357.000000,0.000000e+00,0.000000,1.154250e+05,0.000000e+00,0.000000e+00,0.000000e+00,-413.000000,0.000000e+00,0.000000
75%,-469.000000,0.00000,330.000000,0.000000,0.000000e+00,0.000000,2.700000e+05,7.182000e+01,0.000000e+00,0.000000e+00,-34.000000,0.000000e+00,0.000000
max,0.000000,2792.00000,31198.000000,0.000000,1.314450e+07,6.000000,6.220300e+07,6.457020e+07,2.045370e+06,2.387230e+06,372.000000,5.747620e+07,1.000000


In [37]:
df.shape

(372960, 18)

In [38]:
import dtale
dtale.show(df)

2023-10-01 00:22:32,814 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,816 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,818 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,819 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,820 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,820 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,821 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,822 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,823 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,824 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,825 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,826 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,827 - WA

2023-10-01 00:22:32,981 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,982 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,983 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,984 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,985 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,986 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,988 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,989 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,990 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,991 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,992 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,993 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:32,995 - WA

2023-10-01 00:22:33,062 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,063 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,063 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,064 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,065 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,066 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,066 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,067 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,068 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,068 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,069 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,070 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:33,070 - WA

2023-10-01 00:22:43,783 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,784 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,808 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,809 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,810 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,810 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,811 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,812 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,813 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,814 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,815 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,816 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-01 00:22:43,817 - WA